In [1]:
import os
import sys
from datetime import datetime
import pandas as pd
# import pypinfo
import subprocess
from subprocess import CalledProcessError
import json
from tqdm.notebook import tqdm

from utilities import read_json, save_json, get_datestr_list

df = (
    pd
    .read_csv("../input/check-github-url.csv")
    .query("gh_url_check==1")
)
list_packages = df["pkg"].tolist()
df.head(3)

,pkg,return_code,github_url,homepage,earliest_release,gh_url_check,github
3,jawalang,200.0,https://github.com/Arsybai/jawa-language,https://github.com/Arsybai/jawa-language,2023-04-25T00:08:33,1.0,https://github.com/Arsybai/jawa-language
5,foccoerpy,200.0,https://github.com/GaNiziolek/FoccoERPy,NaN,2023-04-25T13:26:24,1.0,https://github.com/GaNiziolek/FoccoERPy
6,evaluateqa,200.0,https://github.com/MihailSalnikov/EvaluateQA,https://github.com/MihailSalnikov/EvaluateQA,2023-04-26T12:10:16,1.0,https://github.com/MihailSalnikov/EvaluateQA


In [2]:
start_date = "2023-04-24"
# end_date = datetime.today().strftime("%Y-%m-%d")
end_date = "2023-10-31"

dates = get_datestr_list(start_date, end_date)

date_tuples = [(dates[i], dates[i+1]) for i in range(len(dates)-1)]
date_tuples[:3], date_tuples[-3:]

([('2023-04-24', '2023-04-25'),
  ('2023-04-25', '2023-04-26'),
  ('2023-04-26', '2023-04-27')],
 [('2023-10-28', '2023-10-29'),
  ('2023-10-29', '2023-10-30'),
  ('2023-10-30', '2023-10-31')])

In [3]:
output_path = "../output/cache_pypi_downloads_payload_allinstallers"
for package in tqdm(list_packages):
#     if package=="findsystemfontsfilename":
#         continue
    for day in date_tuples:
        start_date, end_date = day
#         print(start_date, end_date)
        filename = f"{start_date}_{end_date}_{package}.json"
        savepath = os.path.join(output_path, filename)
#         print(filename)
    
        # check if file already exist,
        if os.path.exists(savepath):
            # Assert that payload is correct
            try:
                cached_data = read_json(savepath)
            except json.JSONDecodeError as e:
                # Handle the case where JSON is not valid
                print(f"Failed to decode JSON: {e} for {filename}")
                continue
            
            cached_package = cached_data["args"][-2]
            try:
                assert cached_package==package
                continue
            except AssertionError:
                print("Cached wrong package..re-querying..")
                pass
            
        # form shell command
        cmd = [
            "pypinfo", 
            "-j",  # return in JSON format"
            "--start-date",
            f"{start_date}",
            "--end-date",
            f"{end_date}",
            "--all",  # return dl by installers
            f"{package}",  # pkg to query
            "installer",  # return dl by installers
        ]
        
        try:
            data = subprocess.run(cmd, text=True, check=True, capture_output=True)
            json_payload = json.loads(data.stdout)
            json_payload["args"] = data.args
            
            # save payload
            save_json(json_payload, savepath)
            
        except CalledProcessError as e:
            print(f"CalledProcessError - {cmd}")
            print(e.stderr)
            sys.exit(1)

  0%|          | 0/622 [00:00<?, ?it/s]

Failed to decode JSON: Expecting value: line 1 column 1 (char 0) for 2023-09-12_2023-09-13_hoyo-daily-logins-helper.json
Failed to decode JSON: Expecting value: line 1 column 1 (char 0) for 2023-09-25_2023-09-26_cset.json


In [ ]:
!ls